## Cleaning US Dataset

In [ ]:
# Standard libraries
from pathlib import Path

# Third-party libraries
import pandas as pd

def main():
    # Setting US_df as global to use across code
    global US_df

    # Defining file name and path, and reading the CSV
    file_name = "US_quotes.csv"
    output_path = Path.cwd().parent / "data" / "raw" / file_name  # Obtaining the directory
    US_df = pd.read_csv(output_path)

    # Removing " Year Term" from the "Term Length" column and converting to integer
    US_df["Term Length"] = US_df["Term Length"].str.replace(" Year Term", "").astype(int)

    # Removing commas and "$" sign, then convert to float
    US_df["Premium"] = US_df["Premium"].replace({r"\$": "", ",": ""}, regex=True).astype(float)

    # Defining exchange rate (retrieved at time finished collecting final data)
    exchange_rate = 0.74616745 # USD to GBP, source: XE.com, retrieved on 21-04-2025

    # Overwriting the Premium column with GBP values
    US_df["Premium"] = (US_df["Premium"] * exchange_rate).round(2)
    US_df.rename(columns={"Premium": "Premium (£)"}, inplace=True)

    # Adding "Country" column to differentiate after merging
    US_df["Country"] = "US"


    # Printing the DataFrame
    display(US_df)


main()


,Coverage Amount,Term Length,Age,Gender,Is_Smoker,Premium (£),Country
0,100000,10,20,Male,Current user,16.17,US
1,100000,10,20,Male,Current user,16.65,US
2,100000,10,20,Male,Current user,17.56,US
3,100000,10,20,Male,Current user,19.04,US
4,100000,10,20,Male,Current user,20.79,US
...,...,...,...,...,...,...,...
2281,2000000,30,50,Male,Never Used,507.15,US
2282,2000000,30,50,Female,Current user,1017.22,US
2283,2000000,30,50,Female,Current user,1687.08,US
2284,2000000,30,50,Female,Never Used,305.65,US


In [3]:
def main():
    # Setting UK_df as global to use across code
    global UK_df

    # Defining file name and path, and reading the CSV
    input_file = "UK_quotes.csv"
    output_path = Path.cwd().parent / "data" / "raw" / input_file  # Obtaining the directory
    UK_df = pd.read_csv(output_path)

    # Removing commas and "$" sign, then convert to float
    UK_df["Premium"] = UK_df["Premium"].replace({r"\£": "", ",": ""}, regex=True).astype(float)
    UK_df.rename(columns={"Premium": "Premium (£)"}, inplace=True) 

    # Adding "Country" column to differentiate after merging
    UK_df["Country"] = "UK"


    # Printing the DataFrame
    display(UK_df)
    

main()

,Coverage Amount,Term Length,Age,Gender,Is_Smoker,Premium (£),Country
0,100000,10,20,Male,Smoker,5.00,UK
1,100000,10,20,Male,Smoker,5.01,UK
2,100000,10,20,Male,Smoker,5.01,UK
3,100000,10,20,Male,Smoker,5.18,UK
4,100000,10,20,Male,Smoker,5.48,UK
...,...,...,...,...,...,...,...
11889,350000,25,65,Male,Smoker,623.46,UK
11890,350000,25,65,Male,Smoker,640.32,UK
11891,350000,25,65,Male,Smoker,645.73,UK
11892,350000,25,65,Male,Smoker,711.16,UK


## Merging US and UK Datasets

In [5]:
def main():
    # Clean column names in both DataFrames
    UK_df.columns = UK_df.columns.str.strip()
    US_df.columns = US_df.columns.str.strip()

    quotes_df = pd.concat([US_df, UK_df], axis=0, ignore_index=True)

    file_name = "quotes.csv"
    # Define the file path (adjust the path as needed)
    output_path = Path.cwd().parent / "data" / "clean" / file_name

    # Renaming "Gender" to "Is_Male" and changing the values to 1/0 based on gender
    quotes_df.rename(columns={"Gender": "Is_Male"}, inplace=True)
    quotes_df["Is_Male"] = quotes_df["Is_Male"].apply(lambda x: 1 if x == "Male" else 0)

    # Changing the values in "Is_Smoker" column to 1/0 based on smoking status
    quotes_df["Is_Smoker"] = quotes_df["Is_Smoker"].apply(lambda x: 1 if x in ["Current user", "Smoker"] else 0)
    
    # Renaming "Country" to "Is_UK" and changing to binary variable
    quotes_df.rename(columns={"Country": "Is_UK"}, inplace=True)
    quotes_df["Is_UK"] = quotes_df["Is_UK"].apply(lambda x: 1 if x == "UK" else 0)

    # Displaying final DataFrame before saving to CSV
    display(quotes_df)

    # Saving DataFrame to CSV
    quotes_df.to_csv(output_path, index=False)
    print(f"📁 All done — saved {file_name} file to {output_path}")


main()

,Coverage Amount,Term Length,Age,Is_Male,Is_Smoker,Premium (£),Is_UK
0,100000,10,20,1,1,16.17,0
1,100000,10,20,1,1,16.65,0
2,100000,10,20,1,1,17.56,0
3,100000,10,20,1,1,19.04,0
4,100000,10,20,1,1,20.79,0
...,...,...,...,...,...,...,...
14175,350000,25,65,1,1,623.46,1
14176,350000,25,65,1,1,640.32,1
14177,350000,25,65,1,1,645.73,1
14178,350000,25,65,1,1,711.16,1


📁 All done — saved quotes.csv file to c:\Users\Andrew\Documents\code_projects\Assignments\Y2\Empirical_Project\data\clean\quotes.csv
